In [4]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

import seaborn as sns

%matplotlib inline
%load_ext autoreload
%autoreload 2

sns.set()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
import sys
sys.path.append('../')

import data_utils as util

## Load Demographics  and BASMI data

In [6]:
demo_df = pd.read_excel('../data/demographics and Biologics data.xlsx', index_col=0)

# Get year of birth for estimating age
year_of_birth = pd.DataFrame(demo_df['year of Birth'])

# Subselect and rename some columns
demo_df = demo_df[['patient_gender_id','patient_date_of_diagnosis']]
demo_df.rename(columns={'patient_gender_id': 'gender', 'patient_date_of_diagnosis': 'diagnosis_date'}, inplace=True)

basmi_df = pd.read_excel('../data/clean_basmi.xls', index_col=(0,1)).reset_index(level=1, drop=False)
basmi_df.head()

,Date,CRS,TWS,LSFS,LFS,IMS,BS,Drug
patient_id,,,,,,,,
40,1995-05-09,3,1,6,5,3,3.6,NaN
40,1995-06-01,3,1,8,5,3,4.0,NaN
40,1995-06-12,2,1,5,3,2,2.6,NaN
40,1995-11-02,1,1,3,4,2,2.2,NaN
40,1996-05-02,2,1,4,3,2,2.4,NaN


## Data Pre-processing & Setup

In [7]:
# Add year of birth to BASMI data
df = pd.merge(basmi_df, year_of_birth, left_index=True, right_index=True)

# Convert Drug to binary
df['Drug'] = df['Drug'].notnull()

# Add patient age using year of birth
df['Age'] = df['Date'].dt.year - df['year of Birth']
df.drop('year of Birth', axis=1, inplace=True)

# Bin age into 10 bins
bins = [np.floor(x) for x in np.linspace(df['Age'].values.min(), df['Age'].values.max(), 11)]
labels = range(1,11)
df['Age_cat'] = pd.cut(df['Age'], bins=bins, labels=labels)

df.head()

,Date,CRS,TWS,LSFS,LFS,IMS,BS,Drug,Age,Age_cat
patient_id,,,,,,,,,,
40,1995-05-09,3,1,6,5,3,3.6,False,48,5
40,1995-06-01,3,1,8,5,3,4.0,False,48,5
40,1995-06-12,2,1,5,3,2,2.6,False,48,5
40,1995-11-02,1,1,3,4,2,2.2,False,48,5
40,1996-05-02,2,1,4,3,2,2.4,False,49,5


### Normalize patient timeline

In [8]:
def get_norm_years(dates):
    """
    Different from get_norm_years in data_utils
    
    ->  Calculates normlized timeline using start year as 0 
        and increments for each subsequent year
    
    ->  Originally increments for every year included in study
    
    For example, if patient joined in May 1995, next year increment would
    have been June 1996. Now it will increment on Jan 1996. 
    """
    
    years = [d.year for d in dates]
    start_year = min(years)
    norm_years = [year - start_year for year in years]
    
    return norm_years


df['norm_years'] = df.groupby(['patient_id'])['Date'].transform(get_norm_years)

df.head()

,Date,CRS,TWS,LSFS,LFS,IMS,BS,Drug,Age,Age_cat,norm_years
patient_id,,,,,,,,,,,
40,1995-05-09,3,1,6,5,3,3.6,False,48,5,0
40,1995-06-01,3,1,8,5,3,4.0,False,48,5,0
40,1995-06-12,2,1,5,3,2,2.6,False,48,5,0
40,1995-11-02,1,1,3,4,2,2.2,False,48,5,0
40,1996-05-02,2,1,4,3,2,2.4,False,49,5,1


### Aggregate by year in study

For each patient, calculate aggregated scores for every year in the study

* TODO - generalize this to be able to aggregate for other periods

In [9]:
def agg_by_norm_year(df, agg_dict={}):
    return df.groupby(['patient_id','norm_years']).agg(agg_dict)

# Aggregate the data for each patient by normalized year
# Keep the actual year of each norm_year
agg_dict={'BS': 'mean', 'Age': min, 'Drug': np.any, 'Date': lambda x: x.iloc[0].year}
agg_df = agg_by_norm_year(df,agg_dict).rename(columns={'Date': 'year'})

# Add Gender and Diagnosis Date to aggregated, normalized data
agg_df = demo_df.join(agg_df).reset_index().set_index('patient_id')


# Add time since diagnosis variable
agg_df['time_since_diagnosis'] = agg_df['year'] - agg_df['diagnosis_date'].dt.year
agg_df.drop('diagnosis_date', axis=1, inplace=True)
agg_df.head()

,norm_years,gender,BS,Age,Drug,year,time_since_diagnosis
patient_id,,,,,,,
40,0,Female,3.1,48,False,1995,1.0
40,1,Female,2.4,49,False,1996,2.0
40,2,Female,3.0,50,False,1997,3.0
40,3,Female,3.4,51,False,1998,4.0
40,4,Female,3.3,52,False,1999,5.0


# Implement LMM 

Start with using `Drug` and `norm_years` as the predictors and using `BS` as the outcome. 

Thus, we have:

\begin{equation}
\mathbf{y} = \mathbf{X}\mathbf{\beta} + \mathbf{Z}\mathbf{u} + \mathbf{\epsilon}
\end{equation}

In [31]:
from statsmodels.regression.mixed_linear_model import MixedLM
import statsmodels.formula.api as smf

ids_list = agg_df.index 
patient_ids = ids_list.unique()

n_patients = len(patient_ids)
n_samples = agg_df.shape[0]

ids_to_idx = {p_id:idx for idx,p_id in enumerate(patient_ids)}
samples_idx = list(range(n_samples))

y = agg_df['BS'].values

Start with only modelling time as a fixed effect ("norm_years")

In [47]:
X = agg_df['norm_years'].copy()
lm = MixedLM(endog=y, exog=X.values, groups=ids_list)

result = lm.fit()

print("Model Beta's = ",result.bse)
print('\nModel Summary:\n')
result.summary()

Model Beta's =  [  1.07903582e-03   1.99964639e+00]

Model Summary:



<class 'statsmodels.iolib.summary2.Summary'>
"""
          Mixed Linear Model Regression Results
=========================================================
Model:            MixedLM Dependent Variable: y          
No. Observations: 8472    Method:             REML       
No. Groups:       688     Scale:              0.4044     
Min. group size:  1       Likelihood:         -10199.8592
Max. group size:  31      Converged:          Yes        
Mean group size:  12.3                                   
----------------------------------------------------------
           Coef.   Std.Err.    z     P>|z|  [0.025  0.975]
----------------------------------------------------------
x1          0.042     0.001  38.550  0.000   0.039   0.044
Group RE   14.277     1.272                               
=========================================================

"""

#### Include Drugs/No Drugs

Model the Drug variable as a _fixed effect_ and the normalized timeline as a _random effect_

In [48]:
X = agg_df[['norm_years', 'Drug']].copy()
X['Drug'] = X['Drug'].astype(int, inplace=True)

X_fe = X['Drug']
X_re = X['norm_years']

lm = MixedLM(endog=y, exog=X_fe.values, groups=ids_list, exog_re=X_re.values)

result = lm.fit()

print("Model Beta's = ",result.bse)
print('\nModel Summary:\n')
result.summary()

Model Beta's =  [ 0.14978339  0.00317433]

Model Summary:



<class 'statsmodels.iolib.summary2.Summary'>
"""
          Mixed Linear Model Regression Results
=========================================================
Model:            MixedLM Dependent Variable: y          
No. Observations: 8472    Method:             REML       
No. Groups:       688     Scale:              4.7147     
Min. group size:  1       Likelihood:         -19769.6974
Max. group size:  31      Converged:          Yes        
Mean group size:  12.3                                   
----------------------------------------------------------
           Coef.   Std.Err.    z     P>|z|  [0.025  0.975]
----------------------------------------------------------
x1         -1.111     0.150  -7.417  0.000  -1.405  -0.817
x_re1 RE    0.210     0.007                               
=========================================================

"""

### Figure out the difference to this

Figure out what the difference is

In [49]:
X = agg_df[['norm_years', 'Drug']].copy()
X['Drug'] = X['Drug'].astype(int, inplace=True)

X_fe = X['Drug']
X_re = X['norm_years']

lm = MixedLM(endog=y, exog=X, groups=ids_list)

result = lm.fit()

print("Model Beta's = ",result.bse)
print('\nModel Summary:\n')
result.summary()

Model Beta's =  norm_years    0.001163
Drug          0.033636
Group RE      1.999702
dtype: float64

Model Summary:



<class 'statsmodels.iolib.summary2.Summary'>
"""
          Mixed Linear Model Regression Results
=========================================================
Model:            MixedLM Dependent Variable: y          
No. Observations: 8472    Method:             REML       
No. Groups:       688     Scale:              0.4044     
Min. group size:  1       Likelihood:         -10202.1236
Max. group size:  31      Converged:          Yes        
Mean group size:  12.3                                   
---------------------------------------------------------
               Coef.  Std.Err.   z    P>|z| [0.025 0.975]
---------------------------------------------------------
norm_years      0.041    0.001 35.536 0.000  0.039  0.044
Drug            0.022    0.034  0.646 0.518 -0.044  0.088
Group RE       14.278    1.272                           
=========================================================

"""